In [4]:
import numpy as np
import holoviews as hv; hv.extension('bokeh', logo=False)
import panel as pn;     pn.extension()
from panel.interact import interact

import sympy as sp
from IPython.display import display, Latex, Math

from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main

def format_matrix_with_parentheses(A):
    A_latex = sp.latex(A)
    return A_latex.replace("\\begin{bmatrix}", "\\begin{pmatrix}").replace("\\end{bmatrix}", "\\end{pmatrix}")

%load_ext julia.magic

The julia.magic extension is already loaded. To reload it, use:
  %reload_ext julia.magic


In [6]:
%%julia
using Pkg, Revise
gla_dir = "../GenLinAlgProblems"
Pkg.activate(gla_dir)
using GenLinAlgProblems, LinearAlgebra, RowEchelon, LaTeXStrings, Latexify, Markdown, Printf, SymPy
using SparseArrays

  Activating project at `~/elementary-linear-algebra/GenLinAlgProblems`


<div style="float:center;width:100%;text-align:center;">
<strong style="height:100px;color:darkred;font-size:40px;">Functions of Degenerate Matrices</strong><br>
</div>

# 1. Introduction

In the notebooks [**Functions Of A Matrix**](FunctionsOfAMatrix.ipynb) and [**Functions Of A Matrix Examples**](FunctionsOfAMatrix.ipynb)
we saw how to compute functions of a diagonalizable matrix.

In this notebook, we will explore how to compute functions of a degenerate matrix using its [**Jordan Form**](JordanForm.ipynb)

# 2. Jordan Blocks and Matrix Powers

## 2.1 Definition

<div style="float:left;width:100%;background-color:#F2F5A9;color:black;">

**Definition:** A Jordan block $J$ associated with an eigenvalue $\lambda$ is defined as $J = \lambda I + N$<br>
$\qquad$ where $I$ is the identity matrix, and $N$ is a matrix with zeros everywhere except for ones on the superdiagonal<br>
$\qquad$ (the diagonal immediately above the main diagonal).
</div>

Emphasizing the matrix sizes $n\times n$ and the eigenvalues $\lambda$, a Jordan Block $J_n(\lambda) = \lambda I + N_n$ thus has the following structure:

$\qquad
N_n = \begin{pmatrix}
0 & 1 & 0 & \cdots & 0 \\
0 & 0 & 1 & \cdots & 0 \\
\vdots & \vdots & \ddots & \ddots & \vdots \\
0 & 0 & \cdots & 0 & 1 \\
0 & 0 & \cdots & 0 & 0
\end{pmatrix},\;\;
$ and $\;\; J_n(\lambda) = \begin{pmatrix}
\lambda & 1 & 0 & \cdots & 0 \\
0 & \lambda & 1 & \cdots & 0 \\
\vdots & \vdots & \ddots & \ddots & \vdots \\
0 & 0 & \cdots & \lambda & 1 \\
0 & 0 & \cdots & 0 & \lambda
\end{pmatrix}$

## 2.2 Integer Powers of Degenerate Matrices

### 2.2.1 Powers of $N_n$

The powers of $N_n^m$ are simple to compute. For example,

In [7]:
%%julia
N  = [0 1 0 0; 0 0 1 0; 0 0 0 1; 0 0 0 0]
N2 = N^2
N3 = N2 * N
N4 = N3 * N

py_show( L"N_4 =", N, L"\qquad N_4^2 =", N2, L"\qquad N_4^3 =", N3, L"\qquad N_4^4 =", N4,  inline=true)

<IPython.core.display.Latex object>

This behavior is general: given a matrix $N_n$, we have $N_n^n=0$.

<div style="float:left;width:100%;background-color:#F2F5A9;color:black;">

**Definition:** A matrix $A$ is **nilpotent** if there exists a positive integer $k$ such that $A^k = 0.$<br>
$\qquad$ The smallest such $k$ is called the **nilpotency index** of the matrix.
</div>

Thus, **the matrix $N_n$ has nilpotency index $n$.**

Starting with $N_n^0 = I$, each successive power of $N_n$ shifts non-zero entries upward<br>
$\qquad$ to the next superdiagonal (loosing one entry in the process),<br>
$\qquad$ until the final entry 1 is shifted out at power $n$, leaving the zero matrix.

### 2.2.2 Powers of $J_n(\lambda)$

Using the binomial theorem, the $k$-th power of $J$ can be expressed as

$\qquad J_n(\lambda)^k = (\lambda I + N_n)^k = \sum_{m=0}^k \binom{k}{m}\ \lambda^{k-m} N_n^m,$

$\qquad$ where $\binom{k}{m} = \frac{k!}{m!(k-m)!}$ is the binomial coefficient.

Since $N_n^p = 0$ for $p \geq n$ (since $N_n$ is nilpotent of index $n$), this simplifies to

$\begin{aligned}
\qquad J_n^k & = (\lambda I + N_n)^k = \sum_{m=0}^{n-1} \binom{k}{m}\ \lambda^{k-m} N_n^m \\
&= \lambda^k I + k \lambda^{k-1} N_n + \frac{k(k-1)}{2} \lambda^{k-2} N_n^2 + \cdots + \binom{k}{n-1} \lambda^{k-(n-1)} N_n^{n-1}
\end{aligned}
$

for a total of $n$ terms.

**Examples:**

$\qquad J_2 = \begin{pmatrix}
\lambda & 1 \\
0 & \lambda
\end{pmatrix}, \qquad J_2^2 = \begin{pmatrix}
\lambda^2 & 2\lambda \\
0 & \lambda^2
\end{pmatrix}, \qquad J_2^3 = \begin{pmatrix}
\lambda^3 & 3\lambda^2 \\
0 & \lambda^3
\end{pmatrix}, \;\; \dots \;\;
J_2^k = \lambda^k \begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix}
+ k \lambda^{k-1} \begin{pmatrix}
0 & 1 \\
0 & 0
\end{pmatrix}$
<br><br>

$\qquad J_3 = \begin{pmatrix}
\lambda & 1 & 0 \\
0 & \lambda & 1 \\
0 & 0 & \lambda
\end{pmatrix}, \qquad
J_3^k = \lambda^k \begin{pmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{pmatrix}
+ k \lambda^{k-1} \begin{pmatrix}
0 & 1 & 0 \\
0 & 0 & 1 \\
0 & 0 & 0
\end{pmatrix}
+ \frac{k(k-1)}{2} \lambda^{k-2} \begin{pmatrix}
0 & 0 & 1 \\
0 & 0 & 0 \\
0 & 0 & 0
\end{pmatrix}
$

**Remark:** For the special case $\lambda=0,\;\;$ $J_n(0) = N_n,\;\;$ so $\;\;J_n^k(0) = N^k.$<br>
$\qquad$ For $k \ge n$, $J_n^k = 0$

In [8]:
%%julia
function jordan_block_integer_power(λ, n, k)
    T = typeof(λ)

    # Construct N^m efficiently
    function construct_Nm(m, n)
        if m >= n
            return zeros(T, n, n)  # N^m = 0 for m >= n
        end
        spdiagm(0 => zeros(T, n), m => fill(one(T), n-m)) |> Matrix  # Convert to dense
    end

    # Special case: J^0 = I
    if k == 0
        return Matrix(I, n, n)
    end

    # Special case: λ = 0
    if λ == zero(T)
        return construct_Nm(k, n)
    end

    # Compute J^k using the binomial expansion
    Jk = zeros(T, n, n)
    for m in 0:min(k, n-1)  # Only compute terms where N^m is non-zero
        binomial_coeff = factorial(k) ÷ (factorial(m) * factorial(k - m))
        Jk += binomial_coeff * (λ^(k-m)) * construct_Nm(m, n)
    end

    return Jk
end;

In [9]:
%%julia
λ = symbols("λ", real=true)
Jk = [jordan_block_integer_power(λ, 3, k) for k in 1:4]
py_show([arg for k in 1:length(Jk) for arg in (L"\quad J^{%$k} = ", Jk[k])]...)

<IPython.core.display.Latex object>

### 2.2.3 General Case

In general, any matrix can be represented by its Jordan form $A = P J P^{-1}$.

Similar to diagonalizable matrices, we have $A^k = P J^k P^{-1}$, so that the computation of $A^k$<br>
is achieved by computing the power $J^k$ of its Jordan Form.

$J = \begin{pmatrix}
J_1 & 0 & 0 & \cdots & 0 \\
0 & J_2 & 0 & \cdots & 0 \\
0 & 0 & J_3 & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \cdots & J_m
\end{pmatrix}\quad \Rightarrow \quad
J^k =\begin{pmatrix}
J_1^k & 0 & 0 & \cdots & 0 \\
0 & J_2^k & 0 & \cdots & 0 \\
0 & 0 & J_3^k & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \cdots & J_m^k
\end{pmatrix}
$

Here each $J_i$ is a Jordan block of the form $J_n(\lambda)$ for some block size $n\times n$ and some eigenvalue (not necessaily distinct.


In [10]:
%%julia
function integer_power_of_jordan_form(list_of_jordan_blocks, k)
    [ jordan_block_integer_power( block..., k ) for block in list_of_jordan_blocks]
end
function build_block_diagonal(blocks::Vector{<:AbstractMatrix}, T=nothing)
    T = T === nothing ? eltype(blocks[1]) : T
    total_size = sum(size(block, 1) for block in blocks)

    result = zeros(T, total_size, total_size)

    # Fill the block diagonal matrix with the provided blocks
    start_row = 1
    for block in blocks
        block_size = size(block, 1)
        result[start_row:start_row+block_size-1, start_row:start_row+block_size-1] .= block
        start_row += block_size
    end

    return result
end

function jf_integer_power(list_of_jordan_blocks, k)
    build_block_diagonal(integer_power_of_jordan_form(list_of_jordan_blocks, k))
end
;

In [11]:
%%julia
println("Jordan Form with blocks of size 3, 1, 1 and 3, and its 4th power")
result = jf_integer_power([(λ,3),(-1,1),(0,3)], 4)
py_show(L"J =", jf_integer_power([(λ,3),(-1,1),(0,2)], 1), L",\quad J^4 =", result, inline=true, arraystyle=:curly)

Jordan Form with blocks of size 3, 1, 1 and 3, and its 4th power


<IPython.core.display.Latex object>

In [12]:
%%julia
println("Nilpotent matrix example, blocks of size 4 and 2")
println("Since the largest block size is 4, the matrix will reduce to zero in 4 steps")
Jnp = [jf_integer_power([(0,4),(0,2)], i) for i in 1:4]
py_show([arg for k in 1:length(Jnp) for arg in (L"\quad J^{%$k} = ", Jnp[k])]..., inline=true)

Nilpotent matrix example, blocks of size 4 and 2
Since the largest block size is 4, the matrix will reduce to zero in 4 steps


<IPython.core.display.Latex object>

For the full problem we compute $A^k = P J^k P^{-1}$.

In [13]:
%%julia
P,J,P_inv,A = gen_degenerate_matrix( (1,2),(-1,2))
Ak = [P*J^k*P_inv for k in 1:3]
py_show([arg for k in 1:length(Ak) for arg in (L"\quad A^{%$k} = ", Ak[k])]..., inline=true)
@show Ak[3] == A^3;

<IPython.core.display.Latex object>

Ak[3] == A ^ 3 = true

## 2.3 Arbitrary Powers of Degenerate Matrices

The key to defining arbitrary powers of degenerate matrices is to use the generalized binomial theorem for arbitrary $k$ (not necessarily an integer):

$\qquad \binom{k}{m} = \frac{k (k-1) (k-2) \cdots (k-m+1)}{m!}
$

Using the binomial theorem, the $k$-th power of $J$ can be expressed as

$\qquad \left(J^k\right)_{i,j} =
\begin{cases}
\;\binom{k}{j-i}\ \lambda^{k-(j-i)}, & \text{if } j \geq i, \\
\; 0,                                 & \text{otherwise}
\end{cases}$

where $i$ and $j$ are the row and column indices of a Jordan block $J = J_n(\lambda)$.

In [14]:
%%julia
function jordan_block_arbitrary_power(λ, n, k)
    T = promote_type(typeof(λ), typeof(k))  # Promote type for mixed inputs

    # Construct N^m efficiently
    function construct_Nm(m, n)
        if m >= n
            return zeros(T, n, n)  # N^m = 0 for m >= n
        end
        spdiagm(0 => zeros(T, n), m => fill(one(T), n-m)) |> Matrix  # Convert to dense
    end

    # Generalized binomial coefficient
    function generalized_binomial(k, m)
        if m == 0
            return one(T)
        end
        prod(k - l for l in 0:(m-1)) / factorial(m)
    end

    # Special cases
    if k == 0
        return Matrix{T}(I, n, n)
    end
    if λ == zero(T)
        return construct_Nm(k, n)  # Handles nilpotent cases
    end

    # Compute J^k using generalized binomial expansion
    Jk = zeros(T, n, n)
    for m in 0:(n-1)
        binomial_coeff = generalized_binomial(k, m)
        base = λ^(k - m)
        Jk += binomial_coeff * base * construct_Nm(m, n)
    end

    return Jk
end
;

In [15]:
%%julia
kk=2
Jk = [jordan_block_arbitrary_power(Sym(3//5), 3, k) for k in [1, 1//kk]]
py_show(L"J =", Jk[1], L",\quad  J^{\frac{1}{%$kk}} =", Jk[2], inline=true)
@show Jk[1] == Jk[2]^(kk)

kk=3
Jk = [jordan_block_arbitrary_power(Sym(3//5), 3, k) for k in [1, 1//kk]]
py_show(L"J =", Jk[1], L",\quad  J^{\frac{1}{%$kk}} =", Jk[2], inline=true)
@show Jk[1] == Jk[2]^(kk);

<IPython.core.display.Latex object>


Jk[1] == Jk[2] ^ kk = true


<IPython.core.display.Latex object>

Jk[1] == Jk[2] ^ kk = true

In [16]:
%%julia
function arbitrary_power_of_jordan_form(list_of_jordan_blocks, k)
    [jordan_block_arbitrary_power(lmbda, n, k) for (lmbda, n) in list_of_jordan_blocks]
end
# ------------------------------------------------------------------------------------
function jf_arbitrary_power(list_of_jordan_blocks, k)
    build_block_diagonal(arbitrary_power_of_jordan_form(list_of_jordan_blocks, k))
end
;

In [17]:
%%julia
list_of_jordan_blocks = [(Sym(3//5), 3), (λ,1)]; kk = 2

Jk = [jf_arbitrary_power(list_of_jordan_blocks,  1//k) for k in (1, kk)]
P,P_inv = gen_inv_pb( size(Jk[1],1) )

A1      = P*Jk[1]*P_inv
A1_frac = P*Jk[2]*P_inv

py_show( L"J =", Jk[1], L",\quad J^{\frac{1}{%$(kk)}} =", Jk[2], "\n", inline=true, color="blue")

py_show( L"A =", A1, "\n\n", inline=true)
py_show( L"A^{\frac{1}{%$(kk)}} =", A1_frac, "\n", inline=true)

@show A1_frac^kk == A1;

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>


A1_frac ^ kk == A1 = true


## 2.4 Functions of Degenerate Matrices

### 2.4.1 Definition Via  Series Expansions

For a square matrix $A$, functions $f(A)$ can be defined using the matrix's Jordan decomposition:<br>
$\qquad
A = P J P^{-1}, \quad f(A) = P f(J) P^{-1},
$

where $f(J)$ is computed blockwise for each Jordan block.

Functions of matrices can often be computed using their **Taylor series expansions.**<br>
For a function $f(x)$, its Taylor series around $x = 0$ is<br>
$\qquad
f(x) = \sum_{k=0}^\infty \frac{f^{(k)}(0)}{k!} x^k.
$

In [56]:
%%julia

function degenerate_matrix_function(jordan_blocks, func, func_derivatives)
    function jordan_block_function(eigenvalue, size, func, func_derivatives)
        block = zeros(eltype(eigenvalue), size, size)
        N = diagm(0 => zeros(eltype(eigenvalue), size), 1 => ones(eltype(eigenvalue), size-1))  # Nilpotent part

        # Compute diagonal contribution
        f_lambda = func(eigenvalue)
        block += f_lambda * I(size)

        # Add nilpotent contributions
        for k in 1:(size - 1)
            f_derivative = func_derivatives(eigenvalue, k)
            block       += f_derivative * (N^k)
        end

        return block
    end

    # Compute the size of the full matrix
    total_size = sum(block[2] for block in jordan_blocks)

    # Construct the matrix from the Jordan blocks
    J = zeros(eltype(jordan_blocks[1][1]), total_size, total_size)
    start_idx = 1
    for (eigenvalue, size) in jordan_blocks
        J[start_idx:start_idx+size-1, start_idx:start_idx+size-1] = jordan_block_function(eigenvalue, size, func, func_derivatives)
        start_idx += size
    end

    return J
end
;

### 2.4.2 Example: Exponential of a Degenerate Matrix

In [60]:
%%julia
λ1, λ2, t      = symbols("λ_1 λ_2 t")    # Define symbolic eigenvalues
jordan_blocks  = [(λ1*t, 3), (λ2*t, 2)]  # Jordan blocks with symbolic eigenvalues
exp_func       =  x -> exp(x)             # Define the exponential function
exp_derivative = (λ, k) -> exp(λ)

# Compute the symbolic result
result = degenerate_matrix_function(jordan_blocks, exp_func)

z  = result.subs([(λ1,0),(λ2,0)])
z1 = copy(z); z1[:,4:5] .= 0
z2 = copy(z); z2[:,1:3] .= 0
py_show(L"e^{J t}  =", result, L"= ",  L"e^{\lambda_1 t}", z1, L"\;+\;e^{\lambda_2 t}" , z2, inline=true   )

<IPython.core.display.Latex object>

<br><br>
**Let us look at this in more detail.** Let $J_i= \lambda I + N$ be a Jordan matrix consisting of a single block.

* **Taylor Series Expansion:** A matrix function $f(J_i)$ can be expanded using a Taylor series around 
$\lambda$<br><br>
$\qquad
f(J_i) = f(\lambda I + N) = \sum_{k=0}^{m-1} \frac{f^{(k)}(\lambda)}{k!} N^k,
$<br><br>
where $m$ is the size of the block (the nilpotent index ensures $N^m = 0$).<br><br>

* **Function Evaluated at Eigenvalue:** The leading term of the series $f(\lambda I + N)\;\;$ is
$\;\;
f(\lambda) \; I.
$

* **Polynomial from Nilpotent Terms:** The remaining terms involve powers of $N$<br>
multiplied by derivatives of $f$ evaluated at $\lambda$<br><br>
$\qquad
f(J_i) = f(\lambda) \ I + \frac{f'(\lambda)}{1!} \ N + \frac{f''(\lambda)}{2!} \ N^2 + \cdots.
$<br><br>
Since $N^m = 0$, this expansion terminates after $m-1$ terms, resulting in a polynomial in $N$.

* **Final Form:** The function $f(J_i)$ for a Jordan block is
$\;\;
f(J_i) = p_i(\lambda, N),
$<br>
where $p_i$ is a polynomial in $N$, with coefficients involving $f$ and its derivatives evaluated at $ \lambda$.

* **Matrix-Wide Behavior:** Applying $f(A) = P\ f(J)\ P^{-1}$ combines these block-wise results,<br>
yielding a matrix whose entries are polynomials in $N$, scaled by $f$ evaluated at each eigenvalue.

* **Why This Happens:**
The result arises because the function is evaluated at the eigenvalue $\lambda$ for the diagonal part $ \lambda I$,<br> and the nilpotent part $N$ contributes a polynomial due to its finite powers $N^k = 0$ for $k \geq m$.

* **Example Exponential Function:**<br>
For $f(x) = e^x,\;\;
e^{J_i} = e^\lambda \left( I + \frac{N}{1!} + \frac{N^2}{2!} + \cdots + \frac{N^{m-1}}{(m-1)!} \right),
$<br>
which is a polynomial in $N$ multiplied by $e^\lambda$.

### 2.4.3 Example: Logarithm of a Degenerate Matrix

The logarithm of a Jordan block $J = \lambda I + N$ is given by<br>
<div style="width:40%;">

$$
\log(J) = \log(\lambda I + N) = \log(\lambda) \ I + \sum_{k=1}^{m-1} \frac{(-1)^{k+1}}{k} \; \frac{N^k}{\lambda^k},
$$

$\qquad$ where $m$ is the size of the block.
</div>

In [63]:
%%julia
# Compute the logarithm using the function
jordan_blocks   = [(λ, 3)]  # Single Jordan block with λ and size 3
log_func        = x->log(x)
log_derivatives = (λ, k) -> (-1)^(k+1) / (k * λ^k)

log_J           = degenerate_matrix_function(jordan_blocks, log_func, log_derivatives)

# Display the result
py_show( L"J = ", jordan_block_integer_power(λ, 3, 1), L"\quad \log(J) = ", log_J, inline=true)

<IPython.core.display.Latex object>

# 3. Special Cases

## 3.1 Projection Matrices

# X Stuff for later

## 1.1 Definition and Examples

For example, the following matrix $A$ is nilpotent with a nilpotency index of 3:

____
Let us write an *is_nilpotent()* function and test it

In [4]:
%%julia
function is_nilpotent(matrix::AbstractMatrix, max_power::Int=size(matrix,1))
    # with larger integer matrices, this is likely to fail!
    powers        = [matrix]  # Initialize list to store intermediate powers
    current_power = matrix

    for k in 2:max_power
        current_power = current_power * matrix
        push!(powers, current_power)
        if norm(current_power) ≈ 0.0  # Check if the matrix is close to zero
            return (true, k, powers)
        end
    end

    return (false, nothing, powers)  # returns true/false, index of nilpotency, matrix powers
end;

In [5]:
%%julia
function tst_3x3(n=3)
    for _ in 1:n
        _,_,_,A = gen_degenerate_matrix(3)
        ok,_ndx,m_list = is_nilpotent( Int.(A) )

        powers = []
        for i in 1:_ndx  push!( powers, l_show(L"\quad A^{%$(i-1)} =", m_list[i])) end
        py_show(powers...)
    end
end
tst_3x3()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [6]:
def visualize_nilpotent_matrix(powers, ok):
    if powers is None: return
    n           = powers[0].shape[0]
    ndx         = len(powers)
    abs_powers  = [np.abs(power) for power in powers]
    # max_val     = max(*[np.max(abs_powers[k]) for k in range(ndx)],1e-6)
    max_val     = max(np.max(abs_powers[0]), 1e-6)
    epsilon     = max_val * 1e-3  # Small fraction of max_val to create a gap
    color_range = (-epsilon, max_val)  # Widen the range to ensure zero is distinct

    heatmaps = {
        k : hv.HeatMap((range(n), range(n), abs_powers[k]))\
                    .opts(xticks=None, yticks=None, title = f"A^{k}")
        for k in range(ndx)
    }

    dmap = hv.DynamicMap(
        lambda i: heatmaps[i], kdims=["Power"]
    ).redim.values(Power=list(range(0, ndx)))

    return dmap.opts( width=300, height=300, colorbar=True, cmap="plasma", clim=color_range,
        tools=["hover"]
    )

def heatmap_of_powers(N):
    maxint = 3 if N < 10 else 1
    _,_,_,A = Main.gen_degenerate_matrix(N, maxint=maxint)

    ok,_,powers = Main.is_nilpotent( A ,N+1)
    return powers,ok
powers,ok = heatmap_of_powers(15)
print(f"For larger matrices, the numerical computations may fail:\nOk numerically? {ok}")
visualize_nilpotent_matrix(powers,ok)

For larger matrices, the numerical computations may fail:
Ok numerically? True


:DynamicMap   [Power]
   :HeatMap   [x,y]   (z)

## 1.2 Eigenvalues and Eigenvectors

### 1.2.1 Eigenvalues

<div style="float:left;width:100%;height:0.8cm;background-color:#F2F5A9;color:black;">

**Theorem:** All **eigenvalues** of a nilpotent matrix are zero.
</div>

Consider a nilpotent matrix $A$ with nilpotency index $k$ and an eigenpair $(\lambda, v)$

$\qquad A v = \lambda v \Rightarrow  A^k v = \lambda^k v$:<br>
$\qquad$ Since $A^k = 0,\;\;$ we therefore have $\;\;\lambda^k v = 0 \Rightarrow \lambda^k v^t v = 0$.<br>
$\qquad$ Since $v \neq 0,\;\; v^t v = \Vert v \Vert^2 \ne 0\;\;\;\therefore \;\;\lambda = 0$.

### 1.2.2 Eigenvectors of Nilpotent Matrices, Jordan Form

The eigenvectors of a nilpotent matrix $A$  correspond to the eigenvalue $\lambda = 0$.<br>
However, $A$ **may be degenerate,** so that there are not enough eigenvectors.

We therefore consider the [**Jordan Form**](JordanForm.ipynb) of $A$:

The **Jordan Form of a nilpotent matrix** $A$ with index $k$ is $A = P J P^{-1},\;\;$ where<br>
* $J$ is the Jordan canonical form of $A$
* $P$ is an invertible matrix consisting of generalized eigenvectors associated with each block of $J$.

Since the eigenvalues are zeros, $J$ consists of nilpotent Jordan blocks of the form<br>
$\qquad J_n =
\begin{pmatrix}
0 & 1 & 0 & \cdots & 0 \\
0 & 0 & 1 & \cdots & 0 \\
0 & 0 & 0 & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & 1 \\
0 & 0 & 0 & \cdots & 0
\end{pmatrix}$

**The Jordan block $J_n$ of size $n \times n$ is nilpotent with index $n$.**

**Example:** Consider a $6 \times 6$ nilpotent matrix $A$ with Jordan form<br>
$\qquad J = \left(\begin{array}{ccc|cc|c}
0 & \color{red}1 & 0 & 0 & 0 & 0 \\
0 & 0 & \color{red}1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\ \hline
0 & 0 & 0 & 0 & \color{red}1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\ \hline
0 & 0 & 0 & 0 & 0 & 0 \\
\end{array}\right)$

The matrix has 3 Jordan blocks $J_3, J_2$ and $J_1$.

In [7]:
%%julia
J = [ 0 1 0 0 0 0; 0 0 1 0 0 0; 0 0 0 0 0 0; 0 0 0 0 1 0; 0 0 0 0 0 0; 0 0 0 0 0 0]
J2 = J * J
J3 = J2 * J
py_show( L"J =", J, L"\qquad J^2 =", J2, L"\qquad J^3 =", J3 )

<IPython.core.display.Latex object>

**Remark:** The largest Jordan block size determines the smallest $k$
such that $𝐴^k = 0$, i.e.,<br>
$\qquad$ **the largest Jordan block size is the index of nilpotency of the matrix.**

In [8]:
%%julia
P,J,P_inv, A = Main.gen_degenerate_matrix(3,2, maxint=4);
display(py_show(L"A =", A));

display( py_show(L"A = P J P^{-1},\;\;", L"\text{ where }\;\; P =",P, L",\quad \text{and }\;\; J =", J ));
@show A == P*J*P_inv;

<IPython.core.display.Latex object>

nothing


<IPython.core.display.Latex object>

A == P * J * P_inv = truenothing


In [9]:
%%julia
println("Since the largest Jordan block in J has size 3x3, A has nilpotent index 3")
A2 = A * A
A3 = A2 * A
py_show( L"A =", A, L"\qquad A^2 =", A2, L"\qquad A^3 =", A3 )


Since the largest Jordan block in J has size 3x3, A has nilpotent index 3

<IPython.core.display.Latex object>

## 2.1 Powers of a Nilpotent Matrix

Since nilpotent matrices have the property that $A^p = 0$ for $p \ge k$, where $k$ is the nilpotent index of $A$,
the only non-zero powers $A^p$ have $p=0, 1, \dots k-1.$

Given the Jordan form decomposition of $A^p = P J^p P^{-1}$, we need to consider the powers of the Jordan blocks.

Each Jordan block $J_n$ satisfies:
$$J^m = 0 \quad \text{for } m \geq \text{size}(J).$$

#### Code Example: Integer Powers of a Nilpotent Matrix
```julia
# Assumed function: construct_nilpotent_matrix
# Returns P, P_inv, J given a list of Jordan block sizes
# Example: P, P_inv, J = construct_nilpotent_matrix([3, 2])

function matrix_powers(A::Matrix, max_power::Int)
    """
    Computes the powers of a matrix A up to a specified maximum power.

    Args:
        A::Matrix: Input matrix.
        max_power::Int: The maximum power to compute.

    Returns:
        Vector of matrices: A^1, A^2, ..., A^max_power.
    """
    powers = [A]
    for k in 2:max_power
        push!(powers, powers[end] * A)
    end
    return powers
end

# Example: Construct a nilpotent matrix A = P J P^-1
blocksizes = [3, 2]
P, P_inv, J = construct_nilpotent_matrix(blocksizes)
A = P * J * P_inv

# Compute integer powers of A
powers = matrix_powers(A, 5)

println("Powers of the nilpotent matrix A:")
for (k, Ak) in enumerate(powers)
    println("A^$k = \\begin{bmatrix} $(join([join(row, \" & \") for row in eachrow(Ak)], \" \\\\ \")) \\end{bmatrix}")
end

#### Generalization to Fractional Powers

For fractional powers, $A^r$, where $r$ is a real number, the computation relies on the Jordan decomposition of $A$<br>
$\qquad A = P J P^{-1}.$

The fractional power $A^r$ is defined as<br>
$\qquad A^r = P J^r P^{-1},$

$\qquad$ where $J^r$ is computed by raising each Jordan block of $J=\lambda I + N$ to the $r$-th power.

For a Jordan block $J$, its fractional power is given by<br>
$\qquad J^r = (\lambda I + N)^r = \sum_{k=0}^{n-1} \binom{r}{k} (\lambda I)^{r-k} N^k.$

Fractional powers of $A$ are then computed by applying the above formulas to each Jordan block and reconstructing<br>
$\qquad A^r = P J^r P^{-1}$

#### Code Example: Fractional Powers of a Nilpotent Matrix

In [95]:
%%julia
# Function to compute fractional powers of a matrix using its Jordan decomposition
function fractional_power(A, r, P, P_inv, J)
    # Function to compute fractional powers of Jordan blocks
    function jordan_block_power(J::Matrix, r::Real)
        n = size(J, 1)
        λ = J[1, 1]  # Eigenvalue of the Jordan block
        result = zeros(eltype(J), n, n)

        for i in 1:n
            for j in i:n
                m = j - i  # Superdiagonal distance
                result[i, j] = rchoose(r, m) * λ^(r - m)
            end
        end

        return result
    end

    # Compute fractional power of Jordan form
    J_r = zeros(eltype(J), size(J))
    current_row = 1
    for blocksize in block_sizes(J)
        J_r[current_row:current_row+blocksize-1, current_row:current_row+blocksize-1] .= 
            jordan_block_power(J[current_row:current_row+blocksize-1, current_row:current_row+blocksize-1], r)
        current_row += blocksize
    end

    return P * J_r * P_inv
end;

<PyCall.jlwrap fractional_power>

# Example: Compute fractional powers of A = P J P^-1

In [ ]:
%%julia
r = 0.5

A_half = fractional_power(A, r, I, I, J)

py_show( L"J^(1/2) =", A_half )

For $\lambda = 2$, $n = 4$, and $r = \frac{1}{2}$:

Jordan block $J=
\begin{pmatrix}
2 & 1 & 0 & 0 \\
0 & 2 & 1 & 0 \\
0 & 0 & 2 & 1 \\
0 & 0 & 0 & 2
\end{pmatrix}
\qquad J^{1/2} = 
\begin{pmatrix}
2^{1/2} &  2^{-1/2} & \frac{1}{2} \ 2^{-3/2} & \frac{1}{6} \ 2^{-5/2} \\
0 & 2^{1/2} & 1 \ 2^{-1/2} & \frac{1}{2} \ 2^{-3/2} \\
0 & 0 & 2^{1/2} & 1 \ 2^{-1/2} \\
0 & 0 & 0 & 2^{1/2}
\end{pmatrix}$


#### Generalization to Arbitrary Powers

For arbitrary real or complex powers, $A^z$ where $z \in \mathbb{C}$, the same approach can be extended using the Jordan decomposition:
$$A^z = P J^z P^{-1}.$$

The challenge is handling branches of fractional and complex exponents, particularly when $\lambda = 0$ or $\lambda \in \mathbb{C}$. These cases require careful selection of branches for the logarithm and exponential functions.


## X.3 Importance of Nilpotent Matrices

Nilpotent matrices are a cornerstone of linear algebra, with significant theoretical and practical implications:

1. **Jordan Canonical Form**:
   - Nilpotent matrices form the building blocks of the Jordan canonical form, which represents any square matrix as a sum of a diagonalizable matrix and a nilpotent matrix. Understanding nilpotent matrices is crucial for analyzing the structure of linear operators.

2. **Differential Equations**:
   - When solving systems of differential equations, nilpotent matrices simplify the computation of matrix exponentials \( e^{tA} \). This is particularly useful for matrices with repeated eigenvalues.

3. **Iterative Algorithms**:
   - In numerical methods and iterative algorithms, nilpotent-like structures arise naturally, particularly in the context of Krylov subspaces.

4. **Decompositions**:
   - Many matrix factorizations, like the LU decomposition and singular value decomposition (SVD), involve submatrices that may exhibit nilpotent behavior.

5. **Theoretical Insights**:
   - The properties of nilpotent matrices (e.g., all eigenvalues are zero, trace and determinant are zero) reveal deeper insights into linear transformations and the behavior of square matrices.
  
Nilpotent matrices are important in various fields of mathematics and science, including:
- Understanding matrix decomposition and eigenvalues.
- Applications in differential equations and dynamical systems.
- Connections to Lie algebras and other abstract algebraic structures.

## Key Properties
1. For a nilpotent matrix \( A \), there exists a smallest positive integer \( k \) (called the **index of nilpotency**) such that \( A^k = 0 \).
2. All eigenvalues of a nilpotent matrix are zero.
3. A nilpotent matrix is always singular (non-invertible).
4. Nilpotent matrices play a key role in the Jordan canonical form.

# Y. Applications
## Application 1: Jordan Canonical Form

## Application 2: Solving Linear Systems

In [26]:
import sympy as sp

def compute_jordan_form(A): return sp.Matrix(A).jordan_form()
_,_,_,A = Main.gen_degenerate_matrix(3,2,1)
A = sp.Matrix(A)
P,J = compute_jordan_form(A)
latex_string = f"""A = {format_matrix_with_parentheses(A)},\\;\\; J = {format_matrix_with_parentheses(J)},\\;\\; P = {format_matrix_with_parentheses(P)}"""
display(Math(latex_string))

A.eigenvals()

<IPython.core.display.Math object>

{0: 6}

In [ ]:
%%julia
# Function to compute the matrix exponential
using LinearAlgebra

function matrix_exponential(A, t)
    return exp(t * A)
end

# Compute the exponential of A for t = 1.0
t = 1.0
exp_result = matrix_exponential(A, t)
exp_result

### Exercises
1. Prove that any strictly upper triangular matrix is nilpotent.
2. Compute the nilpotency index of the following matrix using Julia:
   \[
   C = \begin{bmatrix} 0 & 1 & 2 \\ 0 & 0 & 1 \\ 0 & 0 & 0 \end{bmatrix}
   \]
3. Write a Julia function to verify if a given matrix is nilpotent and determine its nilpotency index.
4. Use Julia to generate a random nilpotent matrix of size \( n = 5 \) and verify its properties.


## 7. Conclusion

### Conclusion
In this notebook, we explored nilpotent matrices using Julia for computations and Python for visualization. These matrices are fundamental in linear algebra and have significant applications in solving linear systems, understanding Jordan canonical forms, and iterative algorithms.

**Key Takeaways**:
- All eigenvalues of a nilpotent matrix are zero.
- Nilpotent matrices simplify computational tasks.
- Python and Julia together provide a powerful framework for exploring advanced linear algebra topics.

Continue experimenting with the exercises and exploring further applications!

In [189]:
def format_matrix_with_parentheses(A):
    A_latex = sp.latex(A)  # Generate LaTeX representation
    return A_latex.replace("\\begin{bmatrix}", "\\begin{pmatrix}").replace("\\end{bmatrix}", "\\end{pmatrix}")
latex_string = f"J = {format_matrix_with_parentheses(J)}, P = {format_matrix_with_parentheses(P)}"

from IPython.display import display, Math
display(Math(latex_string))

<IPython.core.display.Math object>

In [24]:
%%julia
# Example: Nilpotent matrix with blocks of size 2, 3, and 4
P, J, P_inv, A = gen_degenerate_matrix(2, 3, 4)
py_show( L"A = P J P^{-1}, \quad A =", A)
py_show( L"P =", P, L"\quad J =", J, L"\quad P^{-1} =", P_inv)
@show A == P*J*P_inv
@show P*P_inv == I

# Example: Degenerate matrix with blocks (2, 1) and 3
P, J, P_inv, A = gen_degenerate_matrix((2, 1), 3)
py_show( L"A = P J P^{-1} =", A)
py_show( L"\quad P =", P, L"\quad J =", J, L"\quad P^{-1} =", P_inv)
@show A == P*J*P_inv
@show P*P_inv == I;

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>


A == P * J * P_inv = true
P * P_inv == I = true

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>


A == P * J * P_inv = true
P * P_inv == I = true